## 50. データの入手・整形Permalink
News Aggregator Data Setをダウンロードし、以下の要領で学習データ（train.txt），検証データ（valid.txt），評価データ（test.txt）を作成せよ．

ダウンロードしたzipファイルを解凍し，readme.txtの説明を読む．
情報源（publisher）が”Reuters”, “Huffington Post”, “Businessweek”, “Contactmusic.com”, “Daily Mail”の事例（記事）のみを抽出する．
抽出された事例をランダムに並び替える．
抽出された事例の80%を学習データ，残りの10%ずつを検証データと評価データに分割し，それぞれtrain.txt，valid.txt，test.txtというファイル名で保存する．ファイルには，１行に１事例を書き出すこととし，カテゴリ名と記事見出しのタブ区切り形式とせよ（このファイルは後に問題70で再利用する）．
学習データと評価データを作成したら，各カテゴリの事例数を確認せよ．

In [ ]:
import pandas as pd
def sample_data(data):
    valid_data = data.sample(int(len(data)*0.1))
    test_data = data[~data["ID"].isin(valid_data["ID"])].sample(int(len(data)*0.1))
    train_data = data[~data["ID"].isin(valid_data["ID"]) & ~data["ID"].isin(test_data["ID"])]
    return train_data,valid_data,test_data
data = pd.read_csv("./data/NewsAggregatorDataset/newsCorpora.csv",sep="\t",header=None)
data.columns = ["ID","TITLE","URL","PUBLISHER","CATEGORY","STORY","HOSTNAME","TIMESTAMP"]
data["TITLE"] = data["TITLE"].str.lower()
data = data[data["PUBLISHER"].isin(["Reuters", "Huffington Post", "Businessweek", "Contactmusic.com", "Daily Mail"])]
data = data.reset_index(drop=True)
train_data,valid_data,test_data = sample_data(data)
print(train_data.shape)
print(valid_data.shape)
print(test_data.shape)

## 51. 特徴量抽出Permalink
学習データ，検証データ，評価データから特徴量を抽出し，それぞれtrain.feature.txt，valid.feature.txt，test.feature.txtというファイル名で保存せよ． なお，カテゴリ分類に有用そうな特徴量は各自で自由に設計せよ．記事の見出しを単語列に変換したものが最低限のベースラインとなるであろう．

In [ ]:
def text2vec(series:pd.Series,words):
    vectors = []
    for title in series:
        vec = [0 for _index in range(len(words))]
        for word in title.split(" "):
            df = words[words == word]
            if len(df) != 0:
                vec[df.index[0]] = 1
        vectors.append(vec)
    vector_df = pd.DataFrame(vectors)
    vector_df.columns = [f"words_{column}" for column in vector_df.columns]
    return vector_df

def mk_feature_df(data,words):
    train_vector = text2vec(data["TITLE"],words)

    category_vector = pd.get_dummies(data["PUBLISHER"])
    features = pd.concat([data.reset_index(drop=True)[["ID","CATEGORY"]],category_vector.reset_index(drop=True),train_vector.reset_index(drop=True)],axis=1)
    features["CATEGORY"] = features["CATEGORY"].replace({'b':0, 't':1, 'e':2, 'm':3})
    return features

words = pd.Series([word for text in train_data["TITLE"] for word in text.split(" ")])
words = words.drop_duplicates().reset_index()[0]

train_features = mk_feature_df(train_data,words)
valid_features = mk_feature_df(valid_data,words)
test_features = mk_feature_df(test_data,words)

In [ ]:
def mk_XY_data(train_data):
    X_train = train_data.drop(["ID","CATEGORY"],axis=1)
    Y_train = train_data["CATEGORY"]
    return X_train,Y_train
X_train,Y_train = mk_XY_data(train_features)
X_valid,Y_valid = mk_XY_data(valid_features)
X_test,Y_test = mk_XY_data(test_features)

## 52. 学習Permalink
51で構築した学習データを用いて，ロジスティック回帰モデルを学習せよ．

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(max_iter=1500)
lr.fit(X_train, Y_train)

## 53. 予測Permalink
52で学習したロジスティック回帰モデルを用い，与えられた記事見出しからカテゴリとその予測確率を計算するプログラムを実装せよ．

In [ ]:
valid_features = mk_feature_df(valid_data,words)
X_valid,Y_valid = mk_XY_data(valid_features)
lr.predict_proba(X_train)

## 54. 正解率の計測Permalink
52で学習したロジスティック回帰モデルの正解率を，学習データおよび評価データ上で計測せよ

In [ ]:
def get_accuracy(lr,test_data,grand_truth):
    truth = pd.DataFrame(grand_truth)
    truth["pred"] = lr.predict(test_data)
    return len(truth[truth["CATEGORY"] == truth["pred"]]) / len(truth)
print(get_accuracy(lr,X_train,Y_train))
print(get_accuracy(lr,X_valid,Y_valid))
print(get_accuracy(lr,X_test,Y_test))

## 55. 混同行列の作成Permalink
52で学習したロジスティック回帰モデルの混同行列（confusion matrix）を，学習データおよび評価データ上で作成せよ．

In [ ]:
from sklearn.metrics import confusion_matrix
def get_confusion_matrix(test_data,grand_truth):
    cm = confusion_matrix(grand_truth, lr.predict(test_data))
    cm = pd.DataFrame(cm)
    cm.columns = ['b','t','e','m']
    cm.index = ['b','t','e','m']
    return cm
get_confusion_matrix(X_valid,Y_valid)

## 56. 適合率，再現率，F1スコアの計測Permalink
52で学習したロジスティック回帰モデルの適合率，再現率，F1スコアを，評価データ上で計測せよ．カテゴリごとに適合率，再現率，F1スコアを求め，カテゴリごとの性能をマイクロ平均（micro-average）とマクロ平均（macro-average）で統合せよ．

In [ ]:
from sklearn.metrics import precision_score,recall_score,f1_score

Y_pred = lr.predict(X_valid)
print(f"precision macro : {precision_score(Y_valid.values, Y_pred, average='macro')}")
print(f"precision micro : {precision_score(Y_valid.values, Y_pred, average='micro')}")

print(f"recall macro : {recall_score(Y_valid.values, Y_pred, average='macro')}")
print(f"recall micro : {recall_score(Y_valid.values, Y_pred, average='micro')}")

print(f"f1 macro : {f1_score(Y_valid.values, Y_pred, average='macro')}")
print(f"f1 micro : {f1_score(Y_valid.values, Y_pred, average='micro')}")


In [ ]:
words

In [ ]:
weight_df = pd.DataFrame(lr.coef_).T
weight_df.columns = ['b','t','e','m']
for column in weight_df.columns:
    print(column)
    print(words.loc[weight_df[column].sort_values(ascending=False).head(10).index])
    print()
    print(words.loc[weight_df[column].sort_values(ascending=True).head(10).index])
    print()
    print()

## 58. 正則化パラメータの変更Permalink
ロジスティック回帰モデルを学習するとき，正則化パラメータを調整することで，学習時の過学習（overfitting）の度合いを制御できる．異なる正則化パラメータでロジスティック回帰モデルを学習し，学習データ，検証データ，および評価データ上の正解率を求めよ．実験の結果は，正則化パラメータを横軸，正解率を縦軸としたグラフにまとめよ．

In [ ]:
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt

params = [x*0.01 for x in range(1,10)]
y = []
for x in params:
    print(x)
    lr = LogisticRegression(max_iter=1500,C=x)
    lr.fit(X_train, Y_train)
    y.append(get_accuracy(lr,X_valid,Y_valid))
plt.scatter(x=params,y=y)

## 59. ハイパーパラメータの探索Permalink
学習アルゴリズムや学習パラメータを変えながら，カテゴリ分類モデルを学習せよ．検証データ上の正解率が最も高くなる学習アルゴリズム・パラメータを求めよ．また，その学習アルゴリズム・パラメータを用いたときの評価データ上の正解率を求めよ．

In [ ]:
from sklearn.model_selection import GridSearchCV
params = {"C": [10 ** i for i in range(-2, 2)]
         }
model = GridSearchCV(LogisticRegression(max_iter=1500),params)
model.fit(X_train, Y_train)
print(get_accuracy(model,X_train,Y_train))
print(get_accuracy(model,X_valid,Y_valid))
print(get_accuracy(model,X_test,Y_test))